In [ ]:
import os
import numpy as np
import pandas as pd

class AlterationNumericalAttack:
    
    def __init__(self, watermarked_data_path, attack_proportions=None, dataset = 'housing', p=20, perturbed_attribute = 'MEDV', random_seed=10000):

        self.watermarked_data_path = watermarked_data_path
        self.attack_proportions = attack_proportions if attack_proportions is not None else [0.2, 0.4, 0.6, 0.8, 1.0]
        self.p = p
        self.random_seed = random_seed
        self.perturbed_attribute = perturbed_attribute
        self.dataset = dataset
        np.random.seed(self.random_seed)
        
        # 加载水印数据
        self.loaded_results = np.load(self.watermarked_data_path, allow_pickle=True).item()
        self.watermarked_data = self.loaded_results['watermarked_data']
        
    def apply_attack(self, proportion, save_path):

        temp = self.watermarked_data.copy()
        indices = np.random.choice(len(temp), size=int(proportion * len(temp)), replace=False)
        perturb_values = np.random.uniform(-self.p, self.p, size=len(indices))  # 扰动值
        temp.loc[indices, self.perturbed_attribute] += perturb_values
        temp.to_csv(save_path, index=False)
        # print(f"Alteration attack applied with proportion {proportion}. Modified data saved to {attack_file_path}")

    def execute(self, save_path):

        for proportion in self.attack_proportions:
            self.apply_attack(proportion, save_path)



In [ ]:
# test robustness
attack_range = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]

seeds = range(10000, 10020)
dataset = "energy"
watermark_schemes = ['B2Mark', 'TabularMark', 'HeMark', 'NgoMark']
# watermark_schemes = ['TabularMark']
for watermark_scheme in watermark_schemes:
    for p in attack_range:
        for seed in seeds:
            watermarked_data_path = f"dataset/{watermark_scheme}_dataset/{watermark_scheme}_{dataset}_{seed}.npy"
            save_path = f"dataset/{watermark_scheme}_dataset/{watermark_scheme}_{dataset}_{seed}_{p}.csv"
            AlterationNumericalAttack(watermarked_data_path, attack_proportions=[1], dataset=dataset, p=p, random_seed=seed, perturbed_attribute='Y2').execute(save_path)
        
        
        
    

In [ ]:
from watermarking_schemes.B2Mark import B2MarkWatermarkDetection
import pandas as pd


seed_range = range(10000,10020)
attack_range = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]


dataset = "energy"
g = 2
secret_key = "4d3f2e1c"
watermarked_column='Y2'

original_data_path = "dataset/ENB2012_data.csv"
original_data = pd.read_csv(original_data_path)
medv_max = original_data[watermarked_column].max()
medv_min = original_data[watermarked_column].min()


for p in attack_range:
    z_scores = []
    for seed in seed_range:
            file_path = f"dataset/B2Mark_dataset/B2Mark_{dataset}_{seed}_{p}.csv"
            b2Mark_detection = B2MarkWatermarkDetection(dataset=dataset, seed=seed , g=g, secret_key=secret_key, k = 15, columns_of_interest=['X1', 'X2'], watermarked_column='Y2')
            z_score = b2Mark_detection.run_detection(file_path, medv_min, medv_max)   
            z_scores.append(z_score)

    # average z_score
    average_z_score = sum(z_scores) / len(z_scores)
    print(f"attack_range {p}")
    print(f"Average Z-score: {average_z_score}")


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', '..')))

from watermarking_schemes.HeMark import HeMarkWatermarkDetection

seed_range = range(10000,10020)
attack_range = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]

dataset = "energy"

for p in attack_range:
    z_scores = []

    for seed in seed_range:
        file_path = f"dataset/HeMark_dataset/HeMark_{dataset}_{seed}_{p}.csv"
        HeMark_detection = HeMarkWatermarkDetection(dataset=dataset, seed=seed, watermarked_column='Y2')
        z_score = HeMark_detection.run_detection(file_path)  
        
        # print(z_score) 
        z_scores.append(z_score)

    # average z_score
    average_z_score = sum(z_scores) / len(z_scores)
    print(f"attack_range {p}")
    print(f"Average Z-score: {average_z_score}")

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', '..')))

from watermarking_schemes.NgoMark import NgoMarkWatermarkDetection

import pandas as pd

seed_range = range(10000,10020)
attack_range = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]

dataset = "energy"
watermarked_column='Y2'

original_data_path = "dataset/ENB2012_data.csv"
original_data = pd.read_csv(original_data_path)
medv_max = original_data[watermarked_column].max()
medv_min = original_data[watermarked_column].min()

for p in attack_range:
    z_scores = []

    for seed in seed_range:
        file_path = f"dataset/NgoMark_dataset/NgoMark_{dataset}_{seed}_{p}.csv"
        NgoMark_detection = NgoMarkWatermarkDetection(dataset=dataset, seed=seed, b = 25, watermarked_column='Y2')
        z_score = NgoMark_detection.run_detection(file_path, medv_min, medv_max)  
        
        z_scores.append(z_score)

    # average z_score
    average_z_score = sum(z_scores) / len(z_scores)
    print(f"attack_range {p}")
    print(f"Average Z-score: {average_z_score}")

In [ ]:
# TabularMark
import sys
import os
from watermarking_schemes.TabularMark import TabularMarkWatermarkDetection

seed_range = range(10000,10020)
attack_range = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]

dataset = "energy"
watermarked_column='Y2'

original_data_path = "dataset/ENB2012_data.csv"
z_scores = []


for p in attack_range:
    for seed in seed_range:
        file_path = f"dataset/TabularMark_dataset/TabularMark_{dataset}_{seed}_{p}.csv"
        tabularMark_detection = TabularMarkWatermarkDetection(dataset=dataset, seed=seed, origin_file=original_data_path, watermarked_data=file_path, results_file=f"dataset/TabularMark_dataset/TabularMark_{dataset}_{seed}.npy",  n = int(500/2), p = 18, k = 60, primary_key_cols=['X1', 'X2'], watermarked_column='Y2')
        z_score = tabularMark_detection.detect_watermark()      
        z_scores.append(z_score)

    # average z_score
    average_z_score = sum(z_scores) / len(z_scores)
    print(f"attack_range {p}")
    print(f"Average Z-score: {average_z_score}")